<a href="https://colab.research.google.com/github/AritraSarkar1203/Sulpher-Recover-UNIT-/blob/main/SRU_UNIT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consider Both output in one time

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load your SRU dataset
in_df = pd.read_csv("/content/IN_Table.csv")  # columns: x1...x5
out_df = pd.read_csv("/content/OUT_Table.csv") # columns: y1, y2

# Combine input and output dataframes
df = pd.concat([in_df, out_df], axis=1)


X = df.iloc[:, 0:5].values  # Inputs
y = df.iloc[:, 5:7].values  # Outputs

# Scale features for better kNN graph building
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.9 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph, radius_neighbors_graph
from torch_geometric.data import Data
import numpy as np

# Use training data for graph construction and features
x_train_scaled = X_train[:, 0:5]
y_train_data = y_train[:, 0:2]

# Create kNN graph on the training data
A = kneighbors_graph(x_train_scaled, n_neighbors=10, mode='connectivity', include_self=False)

# Convert the tuple of arrays from nonzero() to a single numpy array before creating the tensor
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

# Convert to PyG Data using scaled training features and training labels
x = torch.tensor(x_train_scaled, dtype=torch.float)
y = torch.tensor(y_train_data, dtype=torch.float) # Remove unsqueeze(1)

graph_data = Data(x=x, edge_index=edge_index, y=y)

In [ ]:
import torch.nn as nn
from torch_geometric.nn import TransformerConv

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = TransformerConv(in_channels, hidden_channels)
        self.relu = nn.ReLU()
        self.conv2 = TransformerConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
model=GNNModel(in_channels=5,hidden_channels=32,out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
losses=[]

epochs=500
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs} , Loss: {loss.item():.4f}")

Epoch 100/500 , Loss: 0.6044
Epoch 200/500 , Loss: 0.5491
Epoch 300/500 , Loss: 0.5115
Epoch 400/500 , Loss: 0.4938
Epoch 500/500 , Loss: 0.4833


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

model.eval()
with torch.no_grad():
    predictions = model(graph_data).numpy()
    true = graph_data.y.numpy()
    rmse = mean_squared_error(true, predictions)
    r2 = r2_score(true, predictions)
    print(f"RMSE: {rmse:.4f}, R²: {r2:.4f}")

RMSE: 0.4840, R²: 0.5214


# Consider only 1 output from NN

In [ ]:
# Combine input and output dataframes
df = pd.concat([in_df, out_df], axis=1)


X = df.iloc[:, 0:5].values  # Inputs
y = df.iloc[:, 5:6].values  # Select only the first output column
#y = df.iloc[:, 6:7].values
# Scale features for better kNN graph building
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Use training data for graph construction and features
x_train_scaled = X_train[:, 0:5]
y_train_data = y_train[:, 0] # Select only the first output column from the training data

# Create kNN graph on the training data
A = kneighbors_graph(x_train_scaled, n_neighbors=5, mode='connectivity', include_self=False)

# Convert the tuple of arrays from nonzero() to a single numpy array before creating the tensor
edge_index = torch.tensor(np.array(A.nonzero()), dtype=torch.long)

# Convert to PyG Data using scaled training features and training labels
x = torch.tensor(x_train_scaled, dtype=torch.float)
y = torch.tensor(y_train_data, dtype=torch.float).unsqueeze(1) # Ensure y is a column vector

graph_data = Data(x=x, edge_index=edge_index, y=y)

In [ ]:
import torch.nn as nn
from torch_geometric.nn import TransformerConv

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.relu = nn.ReLU()
        self.conv2 =GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
model=GNNModel(in_channels=5,hidden_channels=32,out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
losses=[]

epochs=1000
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs} , Loss: {loss.item():.4f}")

Epoch 100/1000 , Loss: 0.5421
Epoch 200/1000 , Loss: 0.5082
Epoch 300/1000 , Loss: 0.4944
Epoch 400/1000 , Loss: 0.4846
Epoch 500/1000 , Loss: 0.4749
Epoch 600/1000 , Loss: 0.4669
Epoch 700/1000 , Loss: 0.4630
Epoch 800/1000 , Loss: 0.4604
Epoch 900/1000 , Loss: 0.4580
Epoch 1000/1000 , Loss: 0.4558


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

model.eval()
with torch.no_grad():
    predictions = model(graph_data).numpy()
    true = graph_data.y.numpy()

    # Select the single output column (which is the first column now)
    true_single = true[:, 0]
    predictions_single = predictions[:, 0]

    # Calculate metrics for the single output column
    mse = mean_squared_error(true_single, predictions_single)
    rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE
    r2 = r2_score(true_single, predictions_single)

    print(f"RMSE: {rmse:.4f}, R²: {r2:.4f}")

RMSE: 0.6631, R²: 0.5626
